In [1]:
# Libraries

import json
import numpy
import os
import shutil

In [2]:
# Tune

from ray.air                  import RunConfig
from ray.tune                 import JupyterNotebookReporter
from ray.tune                 import TuneConfig
from ray.tune                 import Tuner
from ray.tune.logger          import CSVLoggerCallback
from ray.tune.schedulers      import ASHAScheduler
from ray.tune.search.bayesopt import BayesOptSearch
from ray.tune.search.hyperopt import HyperOptSearch

from ray import tune

In [3]:
# Code

from src.cnn import raytune as cnn_raytune
from src.cnn import core    as cnn_core
from src.io  import loader  as data_loader

# 1. Setup

In [4]:
# Setup some directory paths.

OUT_SUBFOLDER = 'nbp06-raytune'
RES_SUBFOLDER = ''

CWD = os.getcwd()
OUT = os.path.join(CWD, 'out')
RES = os.path.join(CWD, 'res')

OUT_DATA  = os.path.join(OUT, 'data',  OUT_SUBFOLDER)
RES_NBP04 = os.path.join(OUT, 'data', 'nbp04-feature')
RES_NBP05 = os.path.join(OUT, 'data', 'nbp05-target')

shutil.rmtree(OUT_DATA, ignore_errors = True)
os.makedirs(OUT_DATA, exist_ok = True)

print(f'     Root Directory : {CWD}')
print(f'   Output Directory : {OUT}')
print(f' Resource Directory : {RES}')

     Root Directory : C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis
   Output Directory : C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\out
 Resource Directory : C:\Developer\Workspace\PyCharm\Projects\upolanc-thesis\res


In [5]:
# Device

DEVICE = cnn_core.get_device(only_cpu = False)

Graphic devices : 1
Selected device : cuda


In [6]:
# Lock random

RANDOM_SEED = cnn_core.lock_random(
	generate_seed = True
)

print('Random seed : {}'.format(RANDOM_SEED))

Random seed : 634811121


In [7]:
# Load the inputs and ouputs

tpm_order = data_loader.load_labels(
	filename = os.path.join(RES_NBP05, 'target-order.json')
)

# 2. Raytune

## 2.1 Core Config

In [8]:
# Define basic data parameters

core_config = {
	'random_seed' : RANDOM_SEED,
	'device'      : DEVICE,
	'model_name'  : 'zrimec2020r',
	'epochs'      : 2,
	'expand_dims' : None,

	'split_size' : {
		'valid' : 0.2,
		'test'  : 0.2
	},
	'input' : {
		'channels' : 1,
		'height'   : 4,
		'width'    : 2150,
		'features' : 64
	},
	'output' : {
		'group0' : 'tissue',
		'group1' : 'tissue-mean',
		'length' : 8
	},
	'files' : {
		'sequences' : lambda : data_loader.load_fasta(filename = os.path.join(RES_NBP04, 'features-bp2150.fasta'), to_string = True),
		'frequency' : lambda : data_loader.load_npz(filename = os.path.join(RES_NBP04, 'features-frequency.npz')),
		'stability' : lambda : data_loader.load_npz(filename = os.path.join(RES_NBP04, 'features-stability.npz')),
		'values'    : lambda : data_loader.load_labels(filename = os.path.join(RES_NBP05, 'target-values.json'), to_numpy = True),
		'order'     : lambda : data_loader.load_labels(filename = os.path.join(RES_NBP05, 'target-order.json'))
	}
}

In [9]:
# Ensure correct

core_config['output']['length'] = len(tpm_order[core_config['output']['group0']])

## 2.2 Search Space

In [10]:
# Define search space
# https://docs.ray.io/en/latest/tune/api_docs/search_space.html

# Notes
# batch_size = 256 : a bit too much for my system

param_space = {
	# Dataset
	'dataset/batch_size' : tune.choice([32, 64, 128]),
	# Optimizer
	'optimizer/lr' : tune.choice([1e-5, 1e-4, 1e-3, 1e-2]),
	'optimizer/l2' : tune.choice([0, 0.01, 0.05]),
	# Dropout
	'model/dropout' : tune.choice([0.1, 0.2, 0.3, 0.4]),
	# Convolution
	'model/conv1/filters' : tune.choice([32, 64, 128, 256]),
	'model/conv1/kernel'  : tune.choice([5, 11, 21, 31, 41]),
	'model/conv1/padding' : 0,
	'model/conv2/filters' : tune.choice([32, 64, 128, 256]),
	'model/conv2/kernel'  : tune.choice([5, 11, 21, 31, 41]),
	'model/conv2/padding' : tune.choice([0, 'same']),
	'model/conv3/filters' : tune.choice([32, 64, 128, 256]),
	'model/conv3/kernel'  : tune.choice([5, 11, 21, 31, 41]),
	'model/conv3/padding' : tune.choice([0, 'same']),
	# Max Pooling
	'model/maxpool1/kernel'  : tune.choice([3, 5, 9]),
	'model/maxpool1/padding' : 'same',
	'model/maxpool2/kernel'  : tune.choice([3, 5, 9]),
	'model/maxpool2/padding' : 'same',
	'model/maxpool3/kernel'  : tune.choice([3, 5, 9]),
	'model/maxpool3/padding' : 'same',
	# Dense
	'model/fc1/features' : tune.choice([64, 128, 256, 512]),
	'model/fc2/features' : tune.choice([64, 128, 256, 512])
}

## 2.3 Tune Config

In [11]:
# Create callable trainer with resources

tune_method = tune.with_resources(
	lambda x : cnn_raytune.tune_method(
		tune_config = x,
		core_config = core_config
	),
	{
		'cpu' : 1,
		'gpu' : 1
	}
)

In [12]:
# Create tune configuration

tune_config = TuneConfig(
	metric      = 'valid_loss',
	mode        = 'min',
	search_alg  = None,
	scheduler   = ASHAScheduler(),
	num_samples = 2,
	trial_name_creator    = lambda x : str(x.trial_id),
	trial_dirname_creator = lambda x : str(x.trial_id)
)

In [13]:
# Create running configuration

reporter = JupyterNotebookReporter(
	max_column_length = 32,
	max_progress_rows = 20,
	parameter_columns = [],
)

run_config = RunConfig(
	name              = core_config['model_name'],
	local_dir         = OUT_DATA,
	callbacks         = None,
	log_to_file       = True,
	progress_reporter = reporter
)

In [14]:
# Create tuner

tuner = Tuner(
	tune_method,
	tune_config = tune_config,
	run_config  = run_config,
	param_space = param_space
)

In [15]:
# Run tuner

report = tuner.fit()

2023-01-26 12:59:08,990	INFO worker.py:1538 -- Started a local Ray instance.


Trial name,date,done,episodes_total,experiment_id,experiment_tag,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,training_iteration,trial_id,valid_loss,valid_mae,valid_r2,warmup_time
39bdb_00000,2023-01-26_13-00-34,True,,30033d99df304194a59c532a84289c57,"0_dataset_batch_size=128,model_conv1_filters=32,model_conv1_kernel=21,model_conv2_filters=64,model_conv2_kernel=5,model_conv2_padding=same,model_conv3_filters=32,model_conv3_kernel=11,model_conv3_padding=same,model_dropout=0.3000,model_fc1_features=512,model_fc2_features=512,model_maxpool1_kernel=9,model_maxpool2_kernel=9,model_maxpool3_kernel=5,optimizer_l2=0,optimizer_lr=0.0010",Home,2,127.0.0.1,16484,True,80.324,21.994,80.324,1674738034,0,,2.09538,2,39bdb_00000,2.09175,1.21739,-2.83014,0.00300312
39bdb_00001,2023-01-26_13-03-29,True,,30033d99df304194a59c532a84289c57,"1_dataset_batch_size=64,model_conv1_filters=256,model_conv1_kernel=5,model_conv2_filters=64,model_conv2_kernel=21,model_conv2_padding=0,model_conv3_filters=64,model_conv3_kernel=31,model_conv3_padding=same,model_dropout=0.4000,model_fc1_features=128,model_fc2_features=256,model_maxpool1_kernel=3,model_maxpool2_kernel=3,model_maxpool3_kernel=3,optimizer_l2=0.0500,optimizer_lr=0.0001",Home,2,127.0.0.1,16484,True,174.449,66.5485,174.449,1674738209,0,,1.06481,2,39bdb_00001,0.832707,0.729116,-0.611856,0.00300312


2023-01-26 13:03:30,359	INFO tune.py:762 -- Total run time: 259.99 seconds (259.85 seconds for the tuning loop).


## 2.4 Reports

In [16]:
# Check if there have been errors

if report.errors :
	print('At least one of the trials has failed.')
else :
	print('No errors.')

No errors.


In [17]:
# Print the results as dataframe

dataframe = report.get_dataframe()

print('Shortest training time : {:.2f} seconds'.format(dataframe['time_total_s'].min()))
print(' Longest training time : {:.2f} seconds'.format(dataframe['time_total_s'].max()))
print()

dataframe[['trial_id', 'training_iteration', 'valid_loss', 'valid_r2', 'valid_mae', 'train_loss']]

Shortest training time : 80.32 seconds
 Longest training time : 174.45 seconds



,trial_id,training_iteration,valid_loss,valid_r2,valid_mae,train_loss
0,39bdb_00000,2,2.092,-2.830,1.217,2.095
1,39bdb_00001,2,0.833,-0.612,0.729,1.065


### 2.4.1 VIsualization

In [18]:
print('TODO')

TODO


### 2.4.2 Best

In [19]:
# Display best trial

best = report.get_best_result()

print('Best metric : loss = {: 8.5f} r2 = {: 8.5f}'.format(best.metrics['valid_loss'], best.metrics['valid_r2']))
print('Best config : ')
print(json.dumps(best.config, indent = '\t'))

Best metric : loss =  0.83271 r2 = -0.61186
Best config : 
{
	"dataset/batch_size": 64,
	"optimizer/lr": 0.0001,
	"optimizer/l2": 0.05,
	"model/dropout": 0.4,
	"model/conv1/filters": 256,
	"model/conv1/kernel": 5,
	"model/conv1/padding": 0,
	"model/conv2/filters": 64,
	"model/conv2/kernel": 21,
	"model/conv2/padding": 0,
	"model/conv3/filters": 64,
	"model/conv3/kernel": 31,
	"model/conv3/padding": "same",
	"model/maxpool1/kernel": 3,
	"model/maxpool1/padding": "same",
	"model/maxpool2/kernel": 3,
	"model/maxpool2/padding": "same",
	"model/maxpool3/kernel": 3,
	"model/maxpool3/padding": "same",
	"model/fc1/features": 128,
	"model/fc2/features": 256
}


In [20]:
print('TODO')

TODO


### 2.4.3 Worst

In [21]:
# Display worst trial

worst = report.get_best_result(metric = 'valid_loss', mode = 'max')

print('Worst metric : loss = {: 8.5f} r2 = {: 8.5f}'.format(worst.metrics['valid_loss'], worst.metrics['valid_r2']))
print('Worst config : ')
print(json.dumps(worst.config, indent = '\t'))

Worst metric : loss =  2.09175 r2 = -2.83014
Worst config : 
{
	"dataset/batch_size": 128,
	"optimizer/lr": 0.001,
	"optimizer/l2": 0,
	"model/dropout": 0.3,
	"model/conv1/filters": 32,
	"model/conv1/kernel": 21,
	"model/conv1/padding": 0,
	"model/conv2/filters": 64,
	"model/conv2/kernel": 5,
	"model/conv2/padding": "same",
	"model/conv3/filters": 32,
	"model/conv3/kernel": 11,
	"model/conv3/padding": "same",
	"model/maxpool1/kernel": 9,
	"model/maxpool1/padding": "same",
	"model/maxpool2/kernel": 9,
	"model/maxpool2/padding": "same",
	"model/maxpool3/kernel": 5,
	"model/maxpool3/padding": "same",
	"model/fc1/features": 512,
	"model/fc2/features": 512
}


In [22]:
print('TODO')

TODO


## 2.4 Save

In [23]:
# Save the tuning results

report.get_dataframe().to_csv(
	os.path.join(OUT_DATA, core_config['model_name'] + '-report.csv')
)